In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split,TimeSeriesSplit,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler

from xgboost import XGBRegressor

from tensorflow import keras
from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential



In [26]:
data = pd.read_csv('Google_Stock_Price_Train.csv')
data.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [27]:
data = data['Open'].values
data

array([325.25, 331.27, 329.83, ..., 793.7 , 783.33, 782.75])

In [28]:
data = data.reshape(-1,1)
data

array([[325.25],
       [331.27],
       [329.83],
       ...,
       [793.7 ],
       [783.33],
       [782.75]])

In [29]:
train = data[:data.shape[0]-50]
test = data[len(train):]

In [30]:
scaller = MinMaxScaler()
train_scaled = scaller.fit_transform(train)

In [31]:
X_train = []
y_train = []
timesteps = 50

for i in range(timesteps,train_scaled.shape[0]):
    X_train.append(train_scaled[i-timesteps:i,0])
    y_train.append(train_scaled[i,0])

In [32]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train.shape

(1158, 50)

In [33]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_train.shape

(1158, 50, 1)

In [36]:
model = Sequential()
model.add(LSTM(32,activation='tanh',input_shape=(X_train.shape[1],1),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(4,activation='tanh',return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=150)

Epoch 1/150
37/37 [==============================] - 4s 23ms/step - loss: 0.0788
Epoch 2/150
37/37 [==============================] - 1s 23ms/step - loss: 0.0233
Epoch 3/150
37/37 [==============================] - 1s 24ms/step - loss: 0.0221
Epoch 4/150
37/37 [==============================] - 1s 26ms/step - loss: 0.0197
Epoch 5/150
37/37 [==============================] - 1s 24ms/step - loss: 0.0182
Epoch 6/150
37/37 [==============================] - 1s 24ms/step - loss: 0.0180
Epoch 7/150
37/37 [==============================] - 1s 23ms/step - loss: 0.0172
Epoch 8/150
37/37 [==============================] - 1s 23ms/step - loss: 0.0162
Epoch 9/150
37/37 [==============================] - 1s 25ms/step - loss: 0.0171
Epoch 10/150
37/37 [==============================] - 1s 24ms/step - loss: 0.0134
Epoch 11/150
37/37 [==============================] - 1s 26ms/step - loss: 0.0150
Epoch 12/150
37/37 [==============================] - 1s 25ms/step - loss: 0.0139
Epoch 13/150
37/37 [=====